네이버에서 ETF 종목 추출

In [ ]:
# import sys
import csv
import requests
import json
import pandas as pd
import datetime

today = datetime.date.today().strftime("%y%m%d")
filename = today + "_etf_code.csv"
f = open(filename, "w", encoding="euc-kr", newline="")

url = 'https://finance.naver.com/api/sise/etfItemList.nhn'
json_data = json.loads(requests.get(url).text)
df = pd.json_normalize(json_data['result']['etfItemList'])

df.to_csv(f)
f.close()
# df.to_excel('etf_list_20251030.xlsx')

print("save")


save


기간별 수익률

In [20]:
import csv
import requests
import pandas as pd
from bs4 import BeautifulSoup

def fn_etf_data(code, name):
    url = "http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A"+code+"&cID=&MenuYn=Y&ReportGB=&NewMenuID=11&stkGb=701"
    print(url)
    
    res = requests.get(url)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, 'lxml')    
    #print(res.text)
    #sys.exit()
    
    url_su = "http://comp.fnguide.com/SVO2/xml/Snapshot_all/{}.xml".format(code)
    
    res_su = requests.get(url_su)
    res_su.raise_for_status()
    soup_su = BeautifulSoup(res_su.text, 'xml')  
    # print(soup_su)
    
    try:        
        #s_name = soup.find("price").find("close_val").text.replace(u'\xa0','')
        #s_name = " "
        #s_name = soup.shareholders_type.get_text()
        #/html/body/div[1]/xml/upjong_B/record[1]/item_nm/text()

        s_code = "e=" + code
        s_name = name

        s_price = soup_su.find("close_val").text
        
        s_max = soup_su.find("high52week").text
        s_min = soup_su.find("low52week").text
        
        #기간별 수익률
        s_1m = soup_su.find("change_1month").text
        s_3m = soup_su.find("change_3month").text
        s_6m = soup_su.find("change_6month").text
        s_1y = soup_su.find("change_12month").text        
        s_3y = soup_su.find("change_36month").text
                
        ####################################################
        
        #분배금
        s_b1 = soup.find("div", attrs={"id":"etfDivInfo1"}).find_all("td")[0].text
        s_b2 = soup.find("div", attrs={"id":"etfDivInfo1"}).find_all("td")[1].text
        s_b3 = soup.find("div", attrs={"id":"etfDivInfo1"}).find_all("td")[2].text
        s_b4 = soup.find("div", attrs={"id":"etfDivInfo1"}).find_all("td")[3].text
        #//*[@id="etfDivInfo1"]/table/tbody/tr[1]/td[1]
        # print(s_b1)
        
        #펀드현황 및 분배금
        
        r_data = [s_name, s_code, s_price, s_1m, s_3m, s_6m, s_1y, s_3y, s_max, s_min, s_b1, s_b2, s_b3, s_b4]
        #r_data = [s_name, s_code, s_price, s_1m, s_3m, s_6m, s_1y, s_3y, s_max, s_min]

        print(r_data)
        return r_data
    
    except:
        #s_name = ""
        s_b1 = ""
        s_b2 = ""
        s_b3 = ""
        s_b4 = ""
        
        s_1m = ""
        s_3m = ""
        s_6m = ""
        s_1y = ""
        s_3y = ""
        s_max = ""
        s_min = ""
        
        print(s_name+" error")        


    #print("---------------------------------------")

In [21]:
import datetime

#today = datetime.date.today().strftime("%y%m%d")
today = "251208"

filename = today + "_etf_period.csv"
f = open(filename, "w", encoding="euc-kr", newline="")
writer = csv.writer(f)

title = "종목명,종목코드,현재가,1개월,3개월,6개월,1년,3년,최고가,최저가,최근분배금,지급일자,연분배횟수,배당수익률".split(",")

#print(type(title))
writer.writerow(title)


# 종목
#codes = ['005930', '005380', '012330', '105560', '033780', '000810', '001040', '294870', '237690', '086450', '230360', '183490', '030520', '089030', '095340', '000640', '073490', '010130', '035420', '035720']

#df1 = pd.read_csv('D:/정리 2020/작업/fn가이드/수집종목.csv', encoding="euc-kr")

file_etf = today + "_etf_code.csv"
df_etf = pd.read_csv(file_etf, encoding="euc-kr")
df_etf_select = df_etf.loc[df_etf['marketSum'] > 100000]
codes = df_etf_select['itemcode']
names = df_etf_select['itemname']

#print(codes)
i = int()

for code, name in zip(codes, names):
    #print(code[2:])
    i = i + 1
    if i%100==0 : print(i)
    data = fn_etf_data(code, name)   
    if data :
        #print(code)
        #print(data)        
        writer.writerow(data)

print('save')
f.close()

http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A360750&cID=&MenuYn=Y&ReportGB=&NewMenuID=11&stkGb=701
['TIGER 미국S&P500', 'e=360750', '25,080', '2.98', '12.14', '25.11', '18.19', '100.53', '25,080', '17,636', '60', '2025/10/31', '4', '0.24']
http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A069500&cID=&MenuYn=Y&ReportGB=&NewMenuID=11&stkGb=701
['KODEX 200', 'e=069500', '59,020', '5.50', '35.56', '57.19', '86.63', '104.18', '59,930', '30,428', '429', '2025/04/30', '4', '0.24']
save


In [24]:
import csv
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import json

code = "360750"
url = "http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A"+code+"&cID=&MenuYn=Y&ReportGB=&NewMenuID=11&stkGb=701"
print(url)

res = requests.get(url)
res.raise_for_status()
soup = BeautifulSoup(res.text, 'lxml')    
#print(res.text)

script = soup.find_all("script")[7].text
#print(script)

match = re.search(r'"val01":\s*"설정일",\s*"val02":\s*"([^"]+)"', script)
if match:
    setup_date = match.group(1)
    print('설정일:', setup_date)
else:
    print('설정일을 찾을 수 없습니다.')


if script.get("val01") == "설정일":
    print("설정일:", script.get("val02"))


http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A360750&cID=&MenuYn=Y&ReportGB=&NewMenuID=11&stkGb=701
설정일: 2020/08/06


AttributeError: 'str' object has no attribute 'get'